In [43]:
# import libraries

import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import dotenv
from datetime import datetime
import itertools

In [44]:
ENV_FILKEPATH = "C:/Users/vrousteau410/OneDrive - Sporting Solutions/Documents/Work/Profile/Rating_of_players_test"
if os.path.exists(ENV_FILKEPATH + "/.env"):
    dotenv.load_dotenv(ENV_FILKEPATH + "/.env")
    secrets_file_path = os.getenv("BIGQUERY_SECRETS_FILE")
    project_id = os.getenv("BIGQUERY_PROJECT_ID")
else:
    raise FileExistsError("please ensure pointing at a .env file")

if secrets_file_path is None or project_id is None:
    raise ValueError("necessary environment variables missing")

In [45]:
client = bigquery.Client.from_service_account_json(
    secrets_file_path, project=project_id
)

In [47]:
# import excel file

df = pd.read_excel("Customer Ratings.xlsx")
df['Date added'] = pd.to_datetime(df['Date added'], format="%d/%m/%Y")
df['formatted_date'] = df['Date added'].dt.strftime("%Y-%m-%d %H:%M:%S")

In [48]:
# Get columns we are interested in 

df = df[['formatted_date','Partner','Acc Number']].dropna().reset_index(drop=True)

In [59]:
df.Partner.value_counts()

Partner
Danske    30
OLG       29
Spinfo    11
CSGO       4
3et        1
Name: count, dtype: int64

In [49]:
# Replace each partners by the name of the table where the data is (for each partner)

df['Partner_table'] = df.Partner.replace({
    'Danske': 'spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed',
    'OLG': 'spoke-profile-prod-1.profile_dataset_olg.profile_betflow_processed',
    'Spinfo': 'spoke-profile-prod-1.profile_dataset_spinfixedodds.profile_betflow_processed',
    'CSGO': 'spoke-profile-prod-1.profile_dataset_csgoempire.profile_betflow_processed',
    '3et': 'spoke-profile-prod-1.profile_dataset_triplebet.profile_betflow_processed'
    }, regex=True)

In [50]:
# Get all the information in a list

list_info = []

for i in range(len(df)):
    list_info.append({
        'table': df.Partner_table[i],
        'id_date_pairs': [
            (df['Acc Number'][i], df['formatted_date'][i]),
        ],
    })

In [51]:
list_info

[{'table': 'spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed',
  'id_date_pairs': [(4166309, '2023-08-15 00:00:00')]},
 {'table': 'spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed',
  'id_date_pairs': [(1565061, '2023-08-15 00:00:00')]},
 {'table': 'spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed',
  'id_date_pairs': [(926880, '2023-08-15 00:00:00')]},
 {'table': 'spoke-profile-prod-1.profile_dataset_olg.profile_betflow_processed',
  'id_date_pairs': [('BEDE25227184', '2023-08-15 00:00:00')]},
 {'table': 'spoke-profile-prod-1.profile_dataset_olg.profile_betflow_processed',
  'id_date_pairs': [('BEDE24532052', '2023-08-16 00:00:00')]},
 {'table': 'spoke-profile-prod-1.profile_dataset_olg.profile_betflow_processed',
  'id_date_pairs': [('BEDE25225784', '2023-08-16 00:00:00')]},
 {'table': 'spoke-profile-prod-1.profile_dataset_olg.profile_betflow_processed',
  'id_date_pairs': [('BEDE25019384', '2023-08-16 00:00:0

In [52]:
list_query_add = []

for table_info in list_info:
    table_name = table_info['table']
    id_date_pairs = table_info['id_date_pairs']

    sql_query = f"SELECT stakeAmount FROM {table_name} WHERE "
    sql_conditions = []

    for id_value, date_value in id_date_pairs:
        condition = f"customerId = '{id_value}' AND placedAt <= '{date_value}'"
        sql_conditions.append(condition)

    sql_query += " OR ".join(sql_conditions)

    # Print the generated query
    print("Generated SQL Query:", sql_query)

    # Execute the query and fetch the results
    query_job = client.query(sql_query)
    results = query_job.result()

    # Convert results to a list of dictionaries
    results_list = list(results)

    # Append the results to the list
    list_query_add.append(results_list)

Generated SQL Query: SELECT stakeAmount FROM spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed WHERE customerId = '4166309' AND placedAt <= '2023-08-15 00:00:00'
Generated SQL Query: SELECT stakeAmount FROM spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed WHERE customerId = '1565061' AND placedAt <= '2023-08-15 00:00:00'
Generated SQL Query: SELECT stakeAmount FROM spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed WHERE customerId = '926880' AND placedAt <= '2023-08-15 00:00:00'
Generated SQL Query: SELECT stakeAmount FROM spoke-profile-prod-1.profile_dataset_olg.profile_betflow_processed WHERE customerId = 'BEDE25227184' AND placedAt <= '2023-08-15 00:00:00'
Generated SQL Query: SELECT stakeAmount FROM spoke-profile-prod-1.profile_dataset_olg.profile_betflow_processed WHERE customerId = 'BEDE24532052' AND placedAt <= '2023-08-16 00:00:00'
Generated SQL Query: SELECT stakeAmount FROM spoke-profile-prod-1.profile_d

In [23]:
list_query = []

for table_info in list_info:
    table_name = table_info['table']
    id_date_pairs = table_info['id_date_pairs']

    sql_query = f"SELECT customerId, betVariety, eventSport, result, SP, placedAt, betId, price, potentialSwing, winnings FROM {table_name} WHERE "
    sql_conditions = []

    for id_value, date_value in id_date_pairs:
        condition = f"customerId = '{id_value}' AND placedAt <= '{date_value}'"
        sql_conditions.append(condition)

    sql_query += " OR ".join(sql_conditions)

    # Print the generated query
    print("Generated SQL Query:", sql_query)

    # Execute the query and fetch the results
    query_job = client.query(sql_query)
    results = query_job.result()

    # Convert results to a list of dictionaries
    results_list = list(results)

    # Append the results to the list
    list_query.append(results_list)

Generated SQL Query: SELECT customerId, betVariety, eventSport, result, SP, placedAt, betId, price, potentialSwing, winnings FROM spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed WHERE customerId = '4166309' AND placedAt <= '2023-08-15 00:00:00'
Generated SQL Query: SELECT customerId, betVariety, eventSport, result, SP, placedAt, betId, price, potentialSwing, winnings FROM spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed WHERE customerId = '1565061' AND placedAt <= '2023-08-15 00:00:00'
Generated SQL Query: SELECT customerId, betVariety, eventSport, result, SP, placedAt, betId, price, potentialSwing, winnings FROM spoke-profile-prod-1.profile_dataset_danskespil.profile_betflow_processed WHERE customerId = '926880' AND placedAt <= '2023-08-15 00:00:00'
Generated SQL Query: SELECT customerId, betVariety, eventSport, result, SP, placedAt, betId, price, potentialSwing, winnings FROM spoke-profile-prod-1.profile_dataset_olg.profile_betflow_p

In [24]:
import itertools

In [25]:
column_names = [
    'customerId',
    'betVariety',
    'eventSport',
    'result',
    'SP',
    'placedAt',
    'betId',
    'price',
    'potentialSwing',
    'winnings'
]

In [26]:
merge_data_test_customers = list(
    itertools.chain(*list_query)
)

# Create dataframe with all bets from the 2 partners

df_all_bet_test_customers = pd.DataFrame.from_records(
    merge_data_test_customers
)

df_all_bet_test_customers.columns = pd.Index(column_names)


In [53]:
column_names_add = [
    'stakeAmount'
]

In [54]:
merge_data_test_customers_add = list(
    itertools.chain(*list_query_add)
)

# Create dataframe with all bets from the 2 partners

df_all_bet_test_customers_stakeAmount = pd.DataFrame.from_records(
    merge_data_test_customers_add
)

df_all_bet_test_customers_stakeAmount.columns = pd.Index(column_names_add)

In [55]:
df_all_bet_test_customers_stakeAmount

,stakeAmount
0,10000
1,1000
2,1000
3,1000
4,1000
...,...
121713,225
121714,450
121715,225
121716,1750


In [12]:
merged_df = pd.read_csv('test_customers_bet_data_v2.csv')

In [57]:
merged_df['stakeAmount'] = df_all_bet_test_customers_stakeAmount['stakeAmount']

In [7]:
df_partner_account = df[['Acc Number','Partner']]

In [8]:
df_partner_account['Acc Number'] = df_partner_account['Acc Number'].astype(str)

C:\Users\vrousteau410\AppData\Local\Temp\ipykernel_8832\528818826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_partner_account['Acc Number'] = df_partner_account['Acc Number'].astype(str)


In [9]:
merged_df = pd.merge(df_all_bet_test_customers, df_partner_account, left_on='customerId', right_on='Acc Number', how='left')

In [10]:
merged_df = merged_df.drop(columns= 'Acc Number')

In [58]:
merged_df.to_csv('test_customers_bet_data_v2.csv')

In [29]:
why0_potentialSwing = merged_df[merged_df.potentialSwing == 0].reset_index(drop = True)

In [33]:
why0_potentialSwing.Partner.value_counts()

Partner
OLG       18466
Spinfo     1933
Name: count, dtype: int64

In [36]:
why0_potentialSwing[why0_potentialSwing.Partner == 'Spinfo']

,customerId,betVariety,eventSport,result,SP,placedAt,betId,price,potentialSwing,winnings,Partner
7193,KV226,MULTIPLE,HORSE_RACING,LOSS,34.327250,2022-12-17 16:36:31+00:00,8364107,34.32725,0.0,0.0,Spinfo
7194,KV226,MULTIPLE,HORSE_RACING,LOSS,4.038500,2022-12-17 16:31:22+00:00,8364102,4.03850,0.0,0.0,Spinfo
7195,KV226,MULTIPLE,HORSE_RACING,LOSS,75.519950,2022-12-17 16:39:53+00:00,8364116,75.51995,0.0,0.0,Spinfo
7196,KV226,SINGLE,HORSE_RACING,LOSS,2.157165,2022-12-17 17:00:57+00:00,8364134,2.00000,0.0,0.0,Spinfo
7197,KV226,MULTIPLE,HORSE_RACING,LOSS,20.625000,2023-01-28 16:36:18+00:00,8418561,20.62500,0.0,0.0,Spinfo
...,...,...,...,...,...,...,...,...,...,...,...
9628,B0583,SINGLE,HORSE_RACING,LOSS,19.000000,2021-06-17 17:13:20+00:00,7210784,19.00000,0.0,0.0,Spinfo
9629,B0583,SINGLE,HORSE_RACING,LOSS,12.000000,2021-06-17 16:35:50+00:00,7210572,12.00000,0.0,0.0,Spinfo
9630,B0583,SINGLE,HORSE_RACING,WIN,21.000000,2021-06-17 17:14:07+00:00,7210785,21.00000,0.0,2600.0,Spinfo
9631,B0583,SINGLE,HORSE_RACING,LOSS,2.100000,2022-03-31 11:59:29+00:00,7847372,2.10000,0.0,0.0,Spinfo


In [26]:
one_customer_df = df_all_bet_test_customers[df_all_bet_test_customers.customerId == '4345814'].reset_index(drop = True)
one_customer_df.potentialSwing.value_counts()

potentialSwing
6000.0    18
8500.0    17
7500.0    13
4800.0    12
8000.0    12
          ..
295.5      1
4290.0     1
2860.0     1
2750.0     1
2650.0     1
Name: count, Length: 545, dtype: int64

In [20]:
one_customer_df['stake'] = one_customer_df['potentialSwing'] / one_customer_df['price']

In [22]:
total_stake = one_customer_df.stake.sum()

In [23]:
total_stake

1874558.1087351216

In [39]:
total_win =one_customer_df.winnings.sum()

In [41]:
total_win-total_stake

90062.65126487869

In [21]:
one_customer_df

,Unnamed: 0,customerId,betVariety,eventSport,result,SP,placedAt,betId,price,potentialSwing,winnings,Partner,stake
0,14702,4345814,SINGLE,TENNIS,LOSS,100.00,2023-07-04 18:14:17.450000+00:00,484897843,100.00,20000.0,0.00,Danske,200.0
1,14703,4345814,SINGLE,FOOTBALL,LOSS,23.00,2023-07-04 17:47:34.163000+00:00,484897497,23.00,11500.0,0.00,Danske,500.0
2,14704,4345814,SINGLE,FOOTBALL,CASHED_OUT,60.00,2023-07-04 17:43:48.290000+00:00,484896941,60.00,18000.0,110.40,Danske,300.0
3,14705,4345814,SINGLE,FOOTBALL,CASHED_OUT,2.20,2023-07-04 16:49:36.210000+00:00,484892516,2.20,6600.0,3113.85,Danske,3000.0
4,14706,4345814,SINGLE,TENNIS,WIN,1.72,2023-07-04 21:52:39.302000+00:00,484910114,1.72,2580.0,2580.00,Danske,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,15928,4345814,SINGLE,TENNIS,LOSS,2.65,2023-08-03 14:08:58.309000+00:00,487565035,2.65,2650.0,0.00,Danske,1000.0
1227,15929,4345814,SINGLE,FOOTBALL,LOSS,19.00,2023-08-03 12:05:10.138000+00:00,487565588,19.00,19000.0,0.00,Danske,1000.0
1228,15930,4345814,SINGLE,FOOTBALL,LOSS,9.00,2023-08-03 18:04:37.547000+00:00,487612429,9.00,9000.0,0.00,Danske,1000.0
1229,15931,4345814,SINGLE,TENNIS,CASHED_OUT,4.30,2023-08-03 13:59:11.460000+00:00,487577648,4.30,12900.0,1551.37,Danske,3000.0
